<a href="https://colab.research.google.com/github/otoperalias/Coyuntura/blob/main/clases/Tema3_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Tema 3.2. Desestacionalización (I)
Leer las diapositivas disponibles en el Aula Virtual antes de ejecutar este código.

In [ ]:
#Importamos algunas librerías
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import seaborn as sns

#### La importancia de desestacionalizar

In [ ]:
# Importamos datos de personas ocupadas (miles) de la CNTR
ocup=pd.read_excel("https://github.com/otoperalias/Coyuntura/raw/main/clases/datos/T3_ocupados_cntr.xlsx", index_col=0,parse_dates=True)
# Calculamos tasa de variación intertrimestral
ocup_tv=(ocup/ocup.shift(1)-1)*100

In [ ]:
# Evolución de los ocupados (datos brutos)
fig,ax=plt.subplots()
sns.set_style("whitegrid") # Aquí utilizo el módulo seaborn, como complemento a matplotlib
sns.lineplot(data=ocup_tv.loc["2005":"2016",'ocup_naj'])
sns.despine(left=True, bottom=True)
ax.xaxis.grid(False)
ax.set_ylabel("Tasa var intertrim (%)\n")
ax.set_xlabel("")
ax.set_title("Evolución de los ocupados en España")
ax.axhline(0, color="black")
fig.text(0.1,0,"Fuente: CNTR (INE). Elaboración propia.")
plt.show()

In [ ]:
# Evolución de los ocupados (datos ajustados)
fig,ax=plt.subplots()
sns.set_style("whitegrid") # Aquí utilizo el módulo seaborn, como complemento a matplotlib
sns.lineplot(data=ocup_tv.loc["2005":"2016",'ocup_aj'])
sns.despine(left=True, bottom=True)
ax.xaxis.grid(False)
ax.set_ylabel("Tasa var intertrim (%)\n")
ax.set_xlabel("")
ax.set_title("Evolución de los ocupados: ajustados por estacionalidad y calendario")
ax.axhline(0, color="black")
fig.text(0.1,0,"Fuente: CNTR (INE). Elaboración propia.")
plt.show()

#### Descomposición clásica

In [ ]:
#Importamos datos de pasajeros de aerolíneas (en miles)
series = pd.read_csv('https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/airline-passengers.csv', header=0, index_col=0, parse_dates=True)
series

In [ ]:
# Representar la serie
series.plot()

In [ ]:
# Ejemplo descomposición aditiva
result_ad = sm.tsa.seasonal_decompose(series, model="additive")
result_ad.plot()
plt.show()

In [ ]:
# Obtenemos serie ajustada
serieaj=series.Passengers-result_ad.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers)
ax.plot(serieaj)
plt.show()

In [ ]:
# Ejemplo de descomposición multiplicativa
result = sm.tsa.seasonal_decompose(series, model='multiplicative')
result.plot()
plt.show()

In [ ]:
# Obtenemos serie ajustada
serieajm=series.Passengers/result.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers)
ax.plot(serieajm)
plt.show()

#### Descomposición STL

In [ ]:
# Echar un vistazo a la página de ayuda:
# https://www.statsmodels.org/stable/generated/statsmodels.tsa.seasonal.STL.html#statsmodels.tsa.seasonal.STL

#Ahora aplicamos el nuevo método a la serie de pasajeros de aerolíneas
result = sm.tsa.STL(series).fit()
result.plot()
plt.show()

In [ ]:
# Para reducir la influencia de los valores atípicos, se añade el argumento robust (no siempre es mejor o conveniente):
result = sm.tsa.STL(series, robust=True).fit()
result.plot()
plt.show()

In [ ]:
serieaj_stl=series.Passengers-result.seasonal

In [ ]:
fig,ax=plt.subplots()
ax.plot(series.Passengers, label="Original")
ax.plot(serieaj_stl,label="STL")
ax.plot(serieaj,label="Aditivo")
ax.plot(serieajm,label="Multiplicativo")
ax.legend()
plt.show()

#### Material adicional:

Para profundizar en vuestros conocimientos, echad un vistazo a este gran recurso disponible en la web:  
http://www.chadfulton.com/topics/002-seasonal-adjustment.html

#### Ejemplo con datos de afiliaciones a la Seguridad Social

Fuente:  
https://www.seg-social.es/wps/portal/wss/internet/EstadisticasPresupuestosEstudios/Estadisticas/EST8/2341/2342

Los datos a usar están disponibles aquí:  
https://github.com/otoperalias/Coyuntura/blob/main/clases/datos/series_seg_social.xlsx  
Se pueden importar directamente, sin necesidad de descargarlos en vuestro equipo. Para ello, debéis usar este enlace:  
https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/series_seg_social.xlsx  
No obstante, la primera vez que lo uséis es bueno descargar el archivo y echar un vistazo a los datos y así, entre otras cosas, ver si hay alguna columna para usar como index de la tabla, si esa columna se puede interpretar como fechas por ```pandas```, etc..


In [ ]:
# Importamos los datos
ss=pd.read_excel("https://raw.githubusercontent.com/otoperalias/Coyuntura/main/clases/datos/series_seg_social.xlsx",
                 index_col=3, parse_dates=True)
ss

In [ ]:
ss["afiliados"].plot()

In [ ]:
# Descomposición clásica aditiva
result = sm.tsa.seasonal_decompose(ss["afiliados"], model="additive")
result.plot()
plt.show()

In [ ]:
# Descomposición clásica multiplicativa
result = sm.tsa.seasonal_decompose(ss["afiliados"], model="multiplcative")
result.plot()
plt.show()

In [ ]:
# Descomposición STL
result = sm.tsa.STL(ss["afiliados"], robust=True).fit()
result.plot()
plt.show()

In [ ]:
# Gráfico para ver en qué meses hay más y menos afiliados
result = sm.tsa.seasonal_decompose(ss["afiliados"], model="multiplcative")
fig,ax=plt.subplots()
result.seasonal["2010"].plot(ax=ax,color="grey",linewidth=4) # da igual el año que se elija (siempre igual en la desc. clásica)
ax.axhline(1,color="black")
plt.show()

In [ ]:
# GRÁFICO MÁS PROFESIONAL
# a. Preparamos los datos
import numpy as np
result = sm.tsa.seasonal_decompose(ss["afiliados"], model="multiplcative")
df=(result.seasonal["2010"]-1)*100  # da igual el año que se elija (siempre igual en la desc. clásica)
meses_num=np.arange(0,12)
meses=["Ene","Feb","Mar","Abr","Mayo","Jun","Jul","Ago","Sep","Oct","Nov","Dec"]

In [ ]:
# a. Creamos el gráfico
plt.style.use('seaborn')
fig,ax=plt.subplots()
df.plot.bar(ax=ax,color="tab:red")
ax.set_xticks(meses_num)
ax.set_xticklabels(meses, rotation='horizontal')
ax.axhline(0,color="black")
ax.set_ylabel("Variación porcentual respecto\n a la media anual (%)")
ax.set_xlabel("")
ax.set_title("Variación mensual en la afiliación a la Seguridad Social", size=15, pad=25)
fig.text(0.5,0.9,"Porcentajes de variación respecto a la media anual", size=13,ha="center")
fig.text(0.13,0,"Periodo analizado: 01/2001-10/20.\nFuente: Seguridad Social. Elaboración propia.")
plt.show()